In [9]:
import pandas as pd
import numpy as np
import pymongo
import json

In [27]:
weather_data = pd.read_csv(
    "/home/navvenramesh/DelhiWeatherAPI/data/Weather Data.csv")

In [28]:
weather_data.columns = weather_data.columns.str.replace(' ', '')

In [29]:
columns_to_replace = ['_pressurem', '_precipm',
                      '_heatindexm', '_wgustm', '_windchillm']
for col in columns_to_replace:
    weather_data[col] = weather_data[col].replace(-9999.0, np.nan)

In [30]:

weather_data = weather_data.rename(columns={
    'datetime_utc': 'datetime',
    '_conds': 'conditions',
    '_dewptm': 'dew_point_c',
    '_fog': 'fog',
    '_hail': 'hail',
    '_heatindexm': 'heat_index_c',
    '_hum': 'humidity',
    '_precipm': 'precipitation_mm',
    '_pressurem': 'pressure_mb',
    '_rain': 'rain',
    '_snow': 'snow',
    '_tempm': 'temperature_c',
    '_thunder': 'thunder',
    '_tornado': 'tornado',
    '_vism': 'visibility_km',
    '_wdird': 'wind_dir_degrees',
    '_wdire': 'wind_direction',
    '_wgustm': 'wind_gust_kph',
    '_windchillm': 'wind_chill_c',
    '_wspdm': 'wind_speed_kph'
})
weather_data.head()

,datetime,conditions,dew_point_c,fog,hail,heat_index_c,humidity,precipitation_mm,pressure_mb,rain,snow,temperature_c,thunder,tornado,visibility_km,wind_dir_degrees,wind_direction,wind_gust_kph,wind_chill_c,wind_speed_kph
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,NaN,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,NaN,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,19961101-14:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.0,0.0,North,NaN,NaN,NaN
4,19961101-16:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.2,0.0,North,NaN,NaN,0.0


In [31]:
# Convert datetime column to proper datetime format
weather_data['datetime'] = pd.to_datetime(
    weather_data['datetime'], format='%Y%m%d-%H:%M')

In [32]:
weather_data

,datetime,conditions,dew_point_c,fog,hail,heat_index_c,humidity,precipitation_mm,pressure_mb,rain,snow,temperature_c,thunder,tornado,visibility_km,wind_dir_degrees,wind_direction,wind_gust_kph,wind_chill_c,wind_speed_kph
0,1996-11-01 11:00:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,1996-11-01 12:00:00,Smoke,10.0,0,0,NaN,32.0,NaN,NaN,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,1996-11-01 13:00:00,Smoke,11.0,0,0,NaN,44.0,NaN,NaN,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,1996-11-01 14:00:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.0,0.0,North,NaN,NaN,NaN
4,1996-11-01 16:00:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.2,0.0,North,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100985,2017-04-24 06:00:00,Haze,17.0,0,0,NaN,25.0,NaN,1005.0,0,0,34.0,0,0,4.0,320.0,NW,NaN,NaN,11.1
100986,2017-04-24 09:00:00,Haze,14.0,0,0,NaN,16.0,NaN,1003.0,0,0,38.0,0,0,4.0,320.0,NW,NaN,NaN,22.2
100987,2017-04-24 12:00:00,Haze,12.0,0,0,NaN,14.0,NaN,1002.0,0,0,36.0,0,0,4.0,270.0,West,NaN,NaN,18.5
100988,2017-04-24 15:00:00,Haze,15.0,0,0,NaN,27.0,NaN,1004.0,0,0,32.0,0,0,2.0,320.0,NW,NaN,NaN,3.7


In [81]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['genai_studio']
collection = db['weather_data']

In [91]:
from collections import defaultdict
import datetime
year_str = "1996"

docs = collection.find({
            'datetime': {
                '$gte': datetime.datetime(int(year_str), 1, 1, 0, 0, 0),
                '$lt': datetime.datetime(int(year_str) + 1, 1, 1, 0, 0, 0)
            }}, {
        'datetime': 1,
        'temperature_c': 1,
        '_id': 0 
    }
)

monthly_temps = defaultdict(list)
for doc in docs:
    month = doc['datetime'].month
    monthly_temps[month].append(doc['temperature_c'])

# Calculate max, min, and avg temperatures for each month
monthly_stats = {}
for month, temps in monthly_temps.items():
    print(month, temps)
    max_temp = max(temps)
    min_temp = min(temps)
    avg_temp = sum(temps)
    print(avg_temp)
    monthly_stats[month] = {
        'high_temp': max_temp,
        'avg_temp': avg_temp,
        'min_temp': min_temp
    }

# Print the results
for month in sorted(monthly_stats):
    print(f"Month: {month}")
    print(f"High Temp: {monthly_stats[month]['high_temp']}")
    print(f"Avg Temp: {monthly_stats[month]['avg_temp']}")
    print(f"Min Temp: {monthly_stats[month]['min_temp']}")
    print()

11 [30.0, 28.0, 24.0, 24.0, 23.0, 21.0, 21.0, 21.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 20.0, 22.0, 23.0, 26.0, 28.0, 30.0, 31.0, 31.0, 31.0, 30.0, 27.0, 25.0, 22.0, 21.0, 19.0, 20.0, 19.0, 19.0, 17.0, 17.0, 17.0, 17.0, 17.0, 16.0, 17.0, 19.0, 22.0, 22.0, 26.0, 28.0, 28.0, 29.0, 29.0, 27.0, 26.0, 22.0, 22.0, 21.0, 21.0, 20.0, 20.0, 19.0, 19.0, 19.0, 17.0, 17.0, 18.0, 15.0, 17.0, 20.0, 26.0, 28.0, 28.0, 29.0, 29.0, 29.0, 27.0, 25.0, 23.0, 21.0, 21.0, 18.0, 18.0, 17.0, 24.0, 26.0, 29.0, 30.0, 30.0, 28.0, 29.0, 29.0, 29.0, 28.0, 25.0, 24.0, 20.0, 19.0, 19.0, 19.0, 18.0, 17.0, 16.0, 16.0, 15.0, 15.0, 17.0, 18.0, 19.0, 23.0, 25.0, 27.0, 29.0, 29.0, 30.0, 30.0, 29.0, 25.0, 24.0, 22.0, 20.0, 19.0, 19.0, 19.0, 18.0, 17.0, 17.0, 16.0, 15.0, 15.0, nan, 16.0, 18.0, 21.0, 24.0, 27.0, 28.0, 29.0, 29.0, 28.0, 27.0, 23.0, 21.0, 21.0, 20.0, 20.0, 19.0, 19.0, 18.0, 17.0, 17.0, 16.0, 16.0, 16.0, 15.0, 16.0, 18.0, 21.0, 24.0, 27.0, 28.0, 28.0, 27.0, 25.0, 21.0, 20.0, 19.0, 19.0, 18.0, 17.0, 17.0, 17.0, 1

In [93]:
pipeline = [
    # Match documents within the specified year
    {
        '$match': {
            'datetime': {
                '$gte': datetime.datetime(int(year_str), 1, 1, 0, 0, 0),
                '$lt': datetime.datetime(int(year_str) + 1, 1, 1, 0, 0, 0)
            },
            # Ensure temperature_c exists and is not null
            'temperature_c': {'$exists': True, '$ne': None}
        }
    },
    # Group by year and month, then calculate high, average, and min temperatures
    {
        '$group': {
            '_id': {
                'year': {'$year': '$datetime'},
                'month': {'$month': '$datetime'}
            },
            # '_id': '$datetime',
            'high_temp': {'$max': '$temperature_c'},
            'avg_temp': {'$avg': '$temperature_c'},
            'min_temp': {'$min': '$temperature_c'}
        }
    },
    # Project the results to include year, month, high_temp, avg_temp, and min_temp fields
    {
        '$project': {
            '_id': 0,
            'year': '$_id.year',
            'month': '$_id.month',
            'high_temp': 1,
            'avg_temp': 1,
            'min_temp': 1
        }
    },
    # Sort the results by month
    {
        '$sort': {
            'month': 1
        }
    }
]

# Execute the aggregation pipeline
weather_details = collection.aggregate(pipeline)

# Print the results
for document in weather_details:
    print(document)

{'high_temp': 34.0, 'avg_temp': 19.43894389438944, 'min_temp': 9.0, 'year': 1996, 'month': 11}
{'high_temp': 24.0, 'avg_temp': 13.816793893129772, 'min_temp': 4.0, 'year': 1996, 'month': 12}


In [ ]:
11 [30.0, 28.0, 24.0, 24.0, 23.0, 21.0, 21.0, 21.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0, 20.0, 22.0, 23.0, 26.0, 28.0, 30.0, 31.0, 31.0, 31.0, 30.0, 27.0, 25.0, 22.0, 21.0, 19.0, 20.0, 19.0, 19.0, 17.0, 17.0, 17.0, 17.0, 17.0, 16.0, 17.0, 19.0, 22.0, 22.0, 26.0, 28.0, 28.0, 29.0, 29.0, 27.0, 26.0, 22.0, 22.0, 21.0, 21.0, 20.0, 20.0, 19.0, 19.0, 19.0, 17.0, 17.0, 18.0, 15.0, 17.0, 20.0, 26.0, 28.0, 28.0, 29.0, 29.0, 29.0, 27.0, 25.0, 23.0, 21.0, 21.0, 18.0, 18.0, 17.0, 24.0, 26.0, 29.0, 30.0, 30.0, 28.0, 29.0, 29.0, 29.0, 28.0, 25.0, 24.0, 20.0, 19.0, 19.0, 19.0, 18.0, 17.0, 16.0, 16.0, 15.0, 15.0, 17.0, 18.0, 19.0, 23.0, 25.0, 27.0, 29.0, 29.0, 30.0, 30.0, 29.0, 25.0, 24.0, 22.0, 20.0, 19.0, 19.0, 19.0, 18.0, 17.0, 17.0, 16.0, 15.0, 15.0, nan, 16.0, 18.0, 21.0, 24.0, 27.0, 28.0, 29.0, 29.0, 28.0, 27.0, 23.0, 21.0, 21.0, 20.0, 20.0, 19.0, 19.0, 18.0, 17.0, 17.0, 16.0, 16.0, 16.0, 15.0, 16.0, 18.0, 21.0, 24.0, 27.0, 28.0, 28.0, 27.0, 25.0, 21.0, 20.0, 19.0, 19.0, 18.0, 17.0, 17.0, 17.0, 15.0, 15.0, 15.0, 14.0, 14.0, 15.0, 18.0, 22.0, 24.0, 26.0, 28.0, 29.0, 29.0, 29.0, 27.0, 23.0, 21.0, 18.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 14.0, 14.0, 15.0, 18.0, 19.0, 24.0, 27.0, 28.0, 29.0, 30.0, 28.0, 29.0, 28.0, 24.0, 23.0, 22.0, 20.0, 20.0, 20.0, 19.0, 20.0, 18.0, 17.0, 16.0, 16.0, 16.0, 21.0, 24.0, 26.0, 28.0, 30.0, 30.0, 31.0, 31.0, 30.0, 28.0, 25.0, 24.0, 23.0, 22.0, 22.0, 21.0, 21.0, 19.0, 20.0, 20.0, 18.0, 17.0, 17.0, 19.0, nan, 21.0, 25.0, 25.0, 26.0, 28.0, 34.0, 28.0, 28.0, 27.0, 25.0, 23.0, 22.0, 21.0, 21.0, 21.0, 20.0, 19.0, 21.0, 23.0, 20.0, 20.0, 21.0, 22.0, 25.0, 26.0, 27.0, 28.0, 28.0, 28.0, 27.0, 24.0, 21.0, 19.0, 19.0, 16.0, 15.0, 14.0, 14.0, 14.0, 14.0, 15.0, 22.0, 23.0, 25.0, 26.0, 24.0, 22.0, 19.0, 17.0, 16.0, 15.0, 14.0, 14.0, 13.0, 12.0, 12.0, 11.0, 11.0, 12.0, 15.0, 17.0, 19.0, nan, 24.0,
    25.0, 25.0, 25.0, 23.0, 22.0, 17.0, 17.0, 17.0, 16.0, 15.0, 14.0, 14.0, 11.0, 11.0, 10.0, 11.0, 11.0, 14.0, 18.0, 21.0, 23.0, 25.0, 25.0, 26.0, 25.0, 24.0, 22.0, 18.0, 17.0, 17.0, 17.0, 16.0, 14.0, 14.0, 14.0, 13.0, 12.0, 12.0, 13.0, 13.0, 15.0, 17.0, 22.0, 25.0, 28.0, 25.0, 21.0, 21.0, 20.0, 19.0, 18.0, 17.0, 16.0, 17.0, 15.0, 17.0, 15.0, 16.0, 16.0, 17.0, 19.0, 20.0, 24.0, 24.0, 25.0, 25.0, 25.0, 25.0, 23.0, 19.0, 17.0, 17.0, 16.0, 17.0, 16.0, 16.0, 16.0, 15.0, 17.0, 17.0, 17.0, 17.0, 22.0, 19.0, 22.0, 23.0, 25.0, 25.0, 23.0, 22.0, 21.0, 20.0, 21.0, 20.0, 20.0, 19.0, 19.0, 16.0, 17.0, 16.0, 14.0, 14.0, 14.0, 15.0, 15.0, 16.0, 19.0, 22.0, 23.0, 25.0, 25.0, 25.0, 20.0, 19.0, 19.0, 19.0, 17.0, 17.0, 16.0, 15.0, 13.0, 13.0, 11.0, 11.0, 11.0, 12.0, 12.0, 16.0, 19.0, 23.0, 21.0, 19.0, 16.0, 15.0, 14.0, 15.0, 13.0, 13.0, 13.0, 12.0, 12.0, 12.0, 12.0, 14.0, 14.0, 17.0, 19.0, 20.0, 21.0, 22.0, 23.0, 22.0, 21.0, 19.0, 17.0, 16.0, 15.0, 15.0, 14.0, 14.0, 13.0, 12.0, 12.0, 11.0, 10.0, 10.0, 11.0, 12.0, 13.0, 16.0, 19.0, 21.0, 22.0, 24.0, 24.0, 24.0, 22.0, 19.0, 16.0, 17.0, 16.0, 15.0, 13.0, 14.0, 12.0, 13.0, 11.0, 12.0, 11.0, 11.0, 12.0, 13.0, 15.0, 17.0, 20.0, 22.0, 24.0, 24.0, 25.0, 23.0, 20.0, 17.0, 17.0, 16.0, 15.0, 15.0, 14.0, 14.0, 14.0, 12.0, 12.0, 13.0, 12.0, 16.0, 17.0, nan, 25.0, 25.0, 27.0, 27.0, 27.0, 28.0, 24.0, 21.0, 19.0, 18.0, 17.0, 17.0, 15.0, 15.0, 15.0, 15.0, 14.0, 14.0, 13.0, 14.0, 15.0, 20.0, 23.0, 25.0, 25.0, 25.0, 24.0, 21.0, 18.0, 17.0, 16.0, 16.0, 15.0, 15.0, 14.0, 13.0, 14.0, 13.0, 12.0, 13.0, 15.0, 21.0, 23.0, 25.0, 24.0, 24.0, 23.0, 20.0, 17.0, 15.0, 17.0, 17.0, 13.0, 14.0, 14.0, 12.0, 12.0, 10.0, 23.0, 16.0, 15.0, 12.0, 12.0, nan, 10.0, 10.0, 10.0, 9.0, 9.0, 9.0, 10.0, 12.0, 15.0, 19.0, 21.0, 21.0, 22.0, 22.0, 22.0, 16.0, 14.0, 14.0, 13.0, 12.0, 11.0, 11.0, 11.0, 10.0, 9.0, 10.0]